In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

# Dataroma

https://www.dataroma.com/m/holdings.php?m=psc

https://www.dataroma.com/m/holdings.php?m=LMM

### Target

1. Track holding history automatically.

2. Grab the weights from the table, and transform it into a usable dataframe in R 

In [2]:
import requests
from bs4 import BeautifulSoup
import sys
import codecs
#from selenium import webdriver
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.common.keys import Keys
import requests
import time
from fake_useragent import UserAgent
import csv
import pandas as pd


### Super Investors

https://www.dataroma.com/m/managers.php

e.g.
https://www.dataroma.com/m/holdings.php?m=tp
https://www.dataroma.com/m/hist/p_hist.php?f=tp


https://www.dataroma.com/m/holdings.php?m=AV
https://www.dataroma.com/m/hist/p_hist.php?f=AV

In [6]:

#Use fake UserAgent
def set_header_user_agent():
    user_agent = UserAgent()
    return user_agent.random

#open browser #打開瀏覽器，但不要載入圖片
#options = webdriver.ChromeOptions()
#prefs = {
#    'profile.default_content_setting_values': {
#        'images': 2,
#        #'javascript': 2
#    }
#}
#options.add_experimental_option('prefs', prefs)

#driver = webdriver.Chrome('./chromedriver', chrome_options=options)


#define urls
user_agent = set_header_user_agent()
super_investor_url = 'https://www.dataroma.com/m/managers.php'
res = requests.get(super_investor_url, headers={ 'user-agent': user_agent })

#res
soup = BeautifulSoup(res.text, 'html.parser')

### Get All Investors ID

In [7]:
links = soup.find_all("a")

In [8]:
IDs = []
for link in links:
    #print(link.text, link['href'])
    if "holdings" in link['href']:
        IDs.append(link['href'])
        #print(link['href'])

In [9]:
investor = []
for ID in IDs:
    investor.append(ID.split(sep = '=')[1])
print(investor)

['AIM', 'AV', 'GFT', 'psc', 'LMM', 'HRSVX', 'oaklx', 'fairx', 'ic', 'ARFFX', 'DJCO', 'tci', 'SA', 'cfimx', 'AC', 'tp', 'abc', 'GLRE', 'MAVFX', 'WP', 'AM', 'DODGX', 'YAFFX', 'EL', 'FPPTX', 'ca', 'CCM', 'aq', 'SSHFX', 'oc', 'FEVAX', 'LUK', 'VA', 'CAAPX', 'KB', 'mc', 'oa', 'MPGFX', 'hcmax', 'LLPFX', 'MVALX', 'SAM', 'MFP', 'PI', 'TF', 'DA', 'FFH', 'pzfvx', 'OFALX', 'TORYX', 'jensx', 'SEQUX', 'lmvtx', 'BAUPOST', 'LPC', 'FPACX', 'FS', 'TA', 'MKL', 'GR', 'TVAFX', 'TWEBX', 'WVALX', 'BRK', 'cc']


# Loop: 把Super Investors各季的權重抓下來

1. 依據Super Investor頁面找到的縮寫來跑迴圈

2. 每一個super investor會有一張weighting matrix

3. 每一個matrix的row都是season, col是個股名稱

4. unique_spam是一個dictionary，key是investor的縮寫，value是他的股池

# remark

* 2020.05.14 更新

將權重的名稱簡化，如`BRK.A`, `BRK.B`都會變成`BRK`

如果簡化前和後的權重都存在，則將cell的值相加，再將column name簡化成`BRK`

In [51]:
#define urls
url = "https://www.dataroma.com/m/hist/p_hist.php?f=" #holding history prefix+ID

unique_spam = {} #存dataroma上所有出現過的股票名稱

#對每個投資人跑找權重
for inv in investor:
    investor_url = "https://www.dataroma.com/m/hist/p_hist.php?f="+inv


    user_agent = set_header_user_agent()
    res = requests.get(investor_url, headers={ 'user-agent': user_agent })
    #res
    soup = BeautifulSoup(res.text, 'html.parser')
    
    #拿Table
    table = soup.find_all('table')
    df = pd.read_html(str(table))[0]
    
    #把period當作index
    df.set_index("Period", inplace = True)

    #type(df.iloc[2][1]) #是str
    #stock_name = testrow.split()[0] #先用空格split
    #print(stock_name) #拿到個股名稱

    #拿個股權重    
    #先做一個有各個股票名稱作為col的array
    #所以要先取出所有的個股名稱存起來在一個list裡，再取unique

    stock_names = []
    for i in range(0,df.shape[0]): #在row裡
        for j in range(0,df.shape[1]): #在col裡
            obs = df.iloc[i][j]       # i-th j-th obs
            try:
                stock_name = obs.split()[0] #用空格分隔後的第一個即是個股名稱
                stock_names.append(stock_name)
            except:
                continue
    uni_names = pd.unique(stock_names)
    uni_names.sort()
    #print(uni_names)

    #排序
    uni_names = uni_names[uni_names>="A"]

    print("The following stocks are hold by: "+inv, end = '\n')
    for i in range(0, len(uni_names)):
        print(uni_names[i], end = ' ')
    print("\n")
    unique_spam[inv] = uni_names #把某個投資人的股池放進對應的dict
        
    #創造很多dict
    names = locals()
    for stock_nam in uni_names:
        names["dict_%s" % stock_nam] = {}
    
    
    #assign dictionary
    names = locals()
    for stock_nam in uni_names:
        names["dict_%s" % stock_nam] = {}

        #前是row, 後是col
        for row in range(0, df.shape[0]):    
            for col in range(0, df.shape[1]):
                try: #因為可能碰到NaN
                    if stock_nam == df.iloc[row][col].split()[0]: #如果stock_nam這個string在某個cell裡
                        testrow = df.iloc[row][col] #為了拿到數值部分
                        test_wei = testrow.split()[-3].split('%')
                        test_wei = test_wei[0][-5:]
                        try: #因為可能取到只有個位數的weight
                            test_wei = float(test_wei)
                        except: #那就退一位數
                            test_wei = test_wei[-4:]
                            test_wei = float(test_wei)

                        names["dict_%s" % stock_nam][row] = test_wei#把數值當作value新增至字典，這時key是所在的row
                except:
                    continue
                    
    dict_stock = [] #list of dictionaries
    dict_stock_str = [] #list of dictionary names, are strings

    for stock_nam in uni_names:
        dict_stock.append(names["dict_%s" % stock_nam]) #把dictionary存進list裡面
        dict_stock_str.append(stock_nam) #存的是字串名稱

    #df_weight = pd.DataFrame(dict_stock) #相當於 pd.DataFrame([dict_1, dict_2]) 這樣的寫法
    df_weight = pd.DataFrame.from_dict(dict_stock)
    df_weight = df_weight.T #因為是by row的加進dataframe裡，所以要transpose

    #rename columns
    df_weight.columns = dict_stock_str
    
    #rename
    rownames = df.index.tolist()
    
    #make a dict of row names
    row_name_dict = {}
    for i in range(0, len(rownames)):
        row_name_dict[i] = rownames[i]
    df_weight = pd.DataFrame.rename(df_weight, index= row_name_dict)
    df_weight.head()
    
    #rename columns again:如果遇到"."或者"-"就刪掉
    #遇到同名則合併
    for col in df_weight.columns:
        if "." in col:
            new_col = col.split(sep = '.')[0]
            if new_col in df_weight.columns: #如果去掉標籤的舊名有在原本的colnam中，代表需要加在一起
                df_weight[new_col] = df_weight[new_col]+df_weight[col]
                df_weight.drop(col, axis = 1, inplace=True) #把舊名丟掉
            else: #直接把colnam換掉
                df_weight.rename(columns={col:new_col}, inplace=True) #直接重新命名
        elif "-" in col:
            new_col = col.split(sep = '-')[0]
            if new_col in df_weight.columns: #如果去掉標籤的舊名有在原本的colnam中，代表需要加在一起
                df_weight[new_col] = df_weight[new_col]+df_weight[col]
                df_weight.drop(col, axis = 1, inplace=True) #把舊名丟掉
            else: #直接把colnam換掉
                df_weight.rename(columns={col:new_col}, inplace=True) #直接重新命名
    
    file_name = "super_investors/"+inv+"_weight.csv"
    df_weight.to_csv(file_name, index=True)

The following stocks are hold by: AIM
ABG ACM ADNT ADT AGCO ALK ARRS ASH ATI AVT AXE AXL AXLL AYI BDC BGC BHI BW BWXT CAG CAH CAM CBT CCK COMM CPRI CR CSC CSX CTB CW CYT DAN DBD DRC DXC EMN ENR FLEX FLS FMC FTI GDI GEO GPI GPK GRA GT HAR HII HPE HPQ HSC HUN IGT IM IP IPGP ITT JOY KEX KFY KLXI KMT KS LEA LEN LSKD MGA MHK MOS MRC MTW MU MWA NAV NCR NOMD NOV NSC NWSA ODP OI OIS OSK PAY RHI RL ROC RRD SCS SEE SJM SPLS SPR SPXC TAM TECD TEN TEX TGI TIF TKR TSLA TTM TW-OLD UNF VC VEC WCC WFT WRK XLS ZTO 

The following stocks are hold by: AV
AAP ADS AN ARES AWI BAC BRK.A BRK.B CHEF CHRW CHTR CMPR CPRT CVET DFZ DNOW DSWL DTV FFH FL FWONK GLIBA GRUB HBI HEI IBKR IBM INWK JEF LB LBTYK LTRPA MCO MNRO MSM OUTR RAVN RBA RCII RUE SAM SAVE SD SNE SPB ST TAP TRIP VIAB VMI WFC XPO XRAY Y 

The following stocks are hold by: GFT
ABT ADM AMX AN APD ARCO BAX BP BRK.B BYND CAT CCI CNI CNR COST CSX CX DDAIF DMND ECL EXPD FDX FMX GS JNJ KO KOF LBTYA LBTYK LILA LILAK MCD MRK MSFT MTB PFE PG RSG SGP TM TSCO TT

#rename columns again:如果遇到"."或者"-"就刪掉
#遇到同名則合併
#shorten_colnam_lst = []
for col in df_weight.columns:
    #print(col)
    if "." in col:
        new_col = col.split(sep = '.')[0]
        #shorten_colnam_lst.append(new_col)
        #temp_df = df_weight[col] #把舊名的col挑出來
        if new_col in df_weight.columns: #如果去掉標籤的舊名有在原本的colnam中，代表需要加在一起
            df_weight[new_col] = df_weight[new_col]+df_weight[col]
            df_weight.drop(col, axis = 1, inplace=True) #把舊名丟掉
        else: #直接把colnam換掉
            df_weight.rename(columns={col:new_col}, inplace=True) #直接重新命名
        #df_weight = pd.concat([df_weight, temp_df], axis = 1) #合併
    elif "-" in col:
        new_col = col.split(sep = '-')[0]
        if new_col in df_weight.columns: #如果去掉標籤的舊名有在原本的colnam中，代表需要加在一起
            df_weight[new_col] = df_weight[new_col]+df_weight[col]
            df_weight.drop(col, axis = 1, inplace=True) #把舊名丟掉
        else: #直接把colnam換掉
            df_weight.rename(columns={col:new_col}, inplace=True) #直接重新命名
    #else:
        #shorten_colnam_lst.append(col)
#print(shorten_colnam_lst)
df_weight

## 找出unique的股池

1. 從investors對應的股池，只取聯集

2. 利用set()來做聯集

In [80]:
unique_spam.values()

dict_values([array(['ABG', 'ACM', 'ADNT', 'ADT', 'AGCO', 'ALK', 'ARRS', 'ASH', 'ATI',
       'AVT', 'AXE', 'AXL', 'AXLL', 'AYI', 'BDC', 'BGC', 'BHI', 'BW',
       'BWXT', 'CAG', 'CAH', 'CAM', 'CBT', 'CCK', 'COMM', 'CPRI', 'CR',
       'CSC', 'CSX', 'CTB', 'CW', 'CYT', 'DAN', 'DBD', 'DRC', 'DXC',
       'EMN', 'ENR', 'FLEX', 'FLS', 'FMC', 'FTI', 'GDI', 'GEO', 'GPI',
       'GPK', 'GRA', 'GT', 'HAR', 'HII', 'HPE', 'HPQ', 'HSC', 'HUN',
       'IGT', 'IM', 'IP', 'IPGP', 'ITT', 'JOY', 'KEX', 'KFY', 'KLXI',
       'KMT', 'KS', 'LEA', 'LEN', 'LSKD', 'MGA', 'MHK', 'MOS', 'MRC',
       'MTW', 'MU', 'MWA', 'NAV', 'NCR', 'NOMD', 'NOV', 'NSC', 'NWSA',
       'ODP', 'OI', 'OIS', 'OSK', 'PAY', 'RHI', 'RL', 'ROC', 'RRD', 'SCS',
       'SEE', 'SJM', 'SPLS', 'SPR', 'SPXC', 'TAM', 'TECD', 'TEN', 'TEX',
       'TGI', 'TIF', 'TKR', 'TSLA', 'TTM', 'TW-OLD', 'UNF', 'VC', 'VEC',
       'WCC', 'WFT', 'WRK', 'XLS', 'ZTO'], dtype=object), array(['AAP', 'ADS', 'AN', 'ARES', 'AWI', 'BAC', 'BRK.A', 'BRK.B', 'CHEF'

In [72]:
#把dictionary的每一個vlaue放進set裡，就unique
unique_spam_set = set()
for lst in unique_spam.values():
    for val in lst:
        #print(val)
        unique_spam_set.add(val)

In [73]:
unique_spam_set

{'GME',
 'AHONY',
 'CBWBF',
 'CFG',
 'GLT',
 'CRESY',
 'TMDX',
 'SUG',
 'ABN',
 'OKE',
 'TTM',
 'CW',
 'DB',
 'WPX',
 'VR',
 'LMDIA',
 'HEOP',
 'JBLU',
 'MER',
 'TPR',
 'PE',
 'MLHR',
 'PSN',
 'AVLR',
 'NPPKF',
 'GM.WS.B',
 'AMGP',
 'TEX',
 'VIAB',
 'HTZ',
 'UFI',
 'LNCE',
 'HII',
 'GPT',
 'HTCH',
 'MDT',
 'LSKD',
 'RAVN',
 'AVX',
 'ESGR',
 'ABB',
 'TWC',
 'DYN',
 'SFI',
 'GBGS',
 'EIG',
 'IMOS',
 'ABC',
 'NLY',
 'SEIC',
 'UNP',
 'USB',
 'UMPQ',
 'IDT',
 'PIR',
 'NPSNY',
 'CACH',
 'APO',
 'BMC',
 'BURL',
 'BKI',
 'WLK',
 'TECK',
 'CELG',
 'FCNCA',
 'CSC',
 'ANF',
 'ONE',
 'NTR',
 'AGX',
 'HSP',
 'FLR',
 'NTAP',
 'TRW',
 'ARO',
 'UHAL',
 'NRF',
 'TPCO',
 'CTVA',
 'TMUS',
 'MDU',
 'RLGY',
 'ICE',
 'VSTO',
 'RH',
 'MIM',
 'IR',
 'RBC',
 'CARE',
 'GYI',
 'ALNY',
 'CYN',
 'MGI',
 'MOS',
 'NEW',
 'CL',
 'WFT',
 'GFG',
 'HD',
 'DLTR',
 'SAFRY',
 'WEC',
 'NRG',
 'DO',
 'XRAY',
 'KLXI',
 'DDAIF',
 'PETQ',
 'CXPO',
 'DVN',
 'TRH',
 'JEF',
 'SGU',
 'AFI',
 'AVGO',
 'CNC',
 'FFH',
 'VVV',
 'BK',
 

In [74]:
#轉成list, 把"."和之後的str丟掉
unique_spam_lst = list(unique_spam_set)
len(pd.unique(unique_spam_lst))

1905

In [75]:
unique_spam_lst.sort()

In [76]:
unique_spam_lst

['A',
 'AA',
 'AABA',
 'AAIGF',
 'AAL',
 'AAN',
 'AAP',
 'AAPL',
 'AB',
 'ABB',
 'ABBC',
 'ABBV',
 'ABC',
 'ABCW',
 'ABEV',
 'ABFS',
 'ABG',
 'ABH',
 'ABKFQ',
 'ABM',
 'ABN',
 'ABR',
 'ABT',
 'ABV-OLD',
 'ACA',
 'ACBI',
 'ACCL',
 'ACF',
 'ACGL',
 'ACIW',
 'ACM',
 'ACMR',
 'ACN',
 'ACOPF',
 'ACT',
 'ACXM',
 'ADBE',
 'ADCT',
 'ADES',
 'ADI',
 'ADM',
 'ADNT',
 'ADP',
 'ADS',
 'ADSK',
 'ADT',
 'ADTN',
 'AEM',
 'AEO',
 'AEP',
 'AER',
 'AES',
 'AET',
 'AFI',
 'AFL',
 'AFSI',
 'AGCO',
 'AGN',
 'AGNC',
 'AGO',
 'AGP',
 'AGR',
 'AGU',
 'AGX',
 'AHL',
 'AHMIQ',
 'AHONY',
 'AIG',
 'AIG.WS',
 'AIN',
 'AIQ',
 'AIV',
 'AIZ',
 'AJG',
 'AJRD',
 'AKAM',
 'AKBA',
 'AKR',
 'ALA',
 'ALB',
 'ALC',
 'ALD',
 'ALDW',
 'ALEX',
 'ALGT',
 'ALK',
 'ALKS',
 'ALL',
 'ALLE',
 'ALLY',
 'ALNY',
 'ALR',
 'ALRM',
 'ALSN',
 'ALTR',
 'ALX',
 'ALXN',
 'AM-OLD',
 'AMAT',
 'AMBA',
 'AMCX',
 'AMD',
 'AME',
 'AMG',
 'AMGN',
 'AMGP',
 'AMH',
 'AMMD',
 'AMP',
 'AMR',
 'AMT',
 'AMTD',
 'AMV',
 'AMX',
 'AMZN',
 'AN',
 'ANAT',
 'AN

In [77]:
unique_spam_reduced_lst = []
for item in unique_spam_lst:
    if "." in item:
        item = item.split(sep = '.')[0]
    elif "-" in item:
        item = item.split(sep = '-')[0]
    unique_spam_reduced_lst.append(item)
unique_spam_reduced_lst = pd.unique(unique_spam_reduced_lst)

In [78]:
len(unique_spam_reduced_lst)

1877

In [79]:
for item in unique_spam_reduced_lst:
    print(item, end = ' ')

A AA AABA AAIGF AAL AAN AAP AAPL AB ABB ABBC ABBV ABC ABCW ABEV ABFS ABG ABH ABKFQ ABM ABN ABR ABT ABV ACA ACBI ACCL ACF ACGL ACIW ACM ACMR ACN ACOPF ACT ACXM ADBE ADCT ADES ADI ADM ADNT ADP ADS ADSK ADT ADTN AEM AEO AEP AER AES AET AFI AFL AFSI AGCO AGN AGNC AGO AGP AGR AGU AGX AHL AHMIQ AHONY AIG AIN AIQ AIV AIZ AJG AJRD AKAM AKBA AKR ALA ALB ALC ALD ALDW ALEX ALGT ALK ALKS ALL ALLE ALLY ALNY ALR ALRM ALSN ALTR ALX ALXN AM AMAT AMBA AMCX AMD AME AMG AMGN AMGP AMH AMMD AMP AMR AMT AMTD AMV AMX AMZN AN ANAT ANDV ANET ANF ANFGF ANK ANSS ANTM ANV AON APA APC APD APFH APH APL APO APOL AR ARCO ARES AREX ARGKF ARIA ARII ARKAY ARMK ARNC ARO AROC ARRS ARW ASA ASCMA ASH ASPS ASTE AT ATGE ATI ATLS ATMI ATR ATRA ATSG ATUS ATVI ATW AU AUQ AUY AVA AVD AVEO AVGO AVLR AVP AVT AVX AVY AW AWH AWI AXE AXELF AXL AXLL AXP AXS AXTA AY AYE AYI AYX AZN AZO AZUL BA BABA BAC BAESF BAGL BAM BANC BANR BAP BAX BB BBBY BBD BBEP BBT BBU BBY BC BCBP BCC BCE BCEI BCO BCR BDC BDK BDX BEAM BEAV BEC BEE BEN BER BF BG B